In [1]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('data/multipleChoiceResponses.csv', dtype='object')

In [10]:
not_to_say = 'I do not wish to disclose my approximate yearly compensation'
outlier = 'What is your current yearly compensation (approximate $USD)?'

In [20]:
clean_df = df[~((pd.isnull(df['Q9'])) | (df['Q9'] == not_to_say) | (['Q9'] == outlier))]

In [31]:
clean_df = clean_df.drop([0])

In [24]:
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_test, y_train, y_test = train_test_split(clean_df[[c for c in clean_df.columns if c != 'Q9']], clean_df['Q9'])

In [22]:
from sklearn.preprocessing import OneHotEncoder

In [34]:
ohe = OneHotEncoder()

In [40]:
X_train_country = ohe.fit_transform(X_train['Q3'][:, np.newaxis])
X_test_country = ohe.transform(X_test['Q3'][:, np.newaxis])

In [42]:
from sklearn.linear_model import LogisticRegression

In [54]:
lr = LogisticRegression(random_state=42, solver='lbfgs', multi_class='multinomial', max_iter=1000)

In [55]:
lr.fit(X_train_country, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [67]:
categories = ['0-10,000', '10-20,000', '20-30,000', '30-40,000', '40-50,000', '50-60,000', '60-70,000', '70-80,000',
              '80-90,000', '90-100,000', '100-125,000', '125-150,000', '150-200,000', '200-250,000', '250-300,000',
              '300-400,000', '400-500,000', '500,000+']

In [77]:
pd.factorize(y_train, sort=True, order=categories)

/home/ptyshevskyi/envs/loc_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: the 'order' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'order'
  """Entry point for launching an IPython kernel.


(array([ 1,  0,  0, ...,  0, 10,  0]),
 Index(['0-10,000', '10-20,000', '100-125,000', '125-150,000', '150-200,000',
        '20-30,000', '200-250,000', '250-300,000', '30-40,000', '300-400,000',
        '40-50,000', '400-500,000', '50-60,000', '500,000+', '60-70,000',
        '70-80,000', '80-90,000', '90-100,000'],
       dtype='object'))

In [50]:
y_train.astype('category')

18614    10-20,000
13338     0-10,000
8646      0-10,000
12778     0-10,000
3954     70-80,000
           ...    
21071     0-10,000
11921    20-30,000
23642     0-10,000
6857     40-50,000
492       0-10,000
Name: Q9, Length: 11571, dtype: category
Categories (18, object): [0-10,000, 10-20,000, 100-125,000, 125-150,000, ..., 60-70,000, 70-80,000, 80-90,000, 90-100,000]

In [47]:
from sklearn.metrics import accuracy_score

In [56]:
accuracy_score(y_test, lr.predict(X_test_country))

0.3019699326075687

In [57]:
import xgboost

In [63]:
xgb = xgboost.XGBRFClassifier(n_estimators=300)

In [64]:
xgb.fit(X_train_country, y_train)

XGBRFClassifier(base_score=0.5, colsample_bylevel=1, colsample_bynode=0.8,
                colsample_bytree=1, gamma=0, learning_rate=1, max_delta_step=0,
                max_depth=3, min_child_weight=1, missing=None, n_estimators=300,
                n_jobs=1, nthread=None, objective='multi:softprob',
                random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                seed=None, silent=None, subsample=0.8, verbosity=1)

In [65]:
accuracy_score(y_test, xgb.predict(X_test_country))

0.29704510108864696

In [8]:
pd.isnull(df['Q9'])

0        False
1         True
2        False
3        False
4         True
         ...  
23855    False
23856     True
23857    False
23858     True
23859     True
Name: Q9, Length: 23860, dtype: bool